# Demo
首先通过.env文件内的全局变量定义模型调用的API，并通过`load_dotenv`函数读取。

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

调用`langchain`中的 OpenAI 式的接口用 deepseek 的 API 定义 llm。
并且使用`@tool`修饰器定义一个工具并绑定到模型上，模型会**自己判断**回答时是否需要调用工具。

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# 使用 @tool 装饰器定义一个工具
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0,
)

# 将工具绑定到大模型
llm_with_tool = llm.bind_tools([get_weather])


# Run the agent
response = llm_with_tool.invoke("what is the weather in sf")
print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}")
# Content: I'll check the weather in San Francisco for you.
# Tool Calls: [{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_00_y9c9mcggTEuG7FQTfac1mt0Y', 'type': 'tool_call'}]


response = llm_with_tool.invoke("Hello, can you introduce yourself?")
print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}")
# I'm here to help you in a friendly and informative way. What can I assist you with today?
# Tool Calls: []

Content: I'll check the weather in San Francisco for you.
Tool Calls: [{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_00_TzKhSL0BY0qmeYpVMgp7dLVm', 'type': 'tool_call'}]
Content: Hello! I'm an AI assistant designed to help you with various tasks. I can provide information, answer questions, and assist with different types of requests. One of the specific capabilities I have is checking weather information for cities around the world. 

I'm here to help you with whatever you need - whether it's getting weather updates, answering general questions, or assisting with other tasks. How can I help you today?
Tool Calls: []


# 逐步构建
## 1. 定义系统提示符
使用系统提示符定义 agent 的角色和行为。

In [3]:
SYSTEM_PROMPT = """
You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. \
If you can tell from the question that they mean wherever they are, \
use the get_user_location tool to find their location.
"""

## 2. 定义大模型使用的工具
工具可以通过调用你定义的函数，让模型与外部系统交互。工具可以依赖运行时上下文 ，也可以与代理内存交互。

In [4]:
from dataclasses import dataclass
from langchain_core.runnables import RunnableConfig

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# @dataclass 修饰符会自动生成数据类的常用方法： __init__、__repr__、__eq__ 
# @dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# ToolRuntime 是连接 LLM 逻辑 和 后端系统状态 的桥梁。
# 它允许你在工具内部访问那些 LLM 无法生成的系统级数据（如用户身份、配置信息等）。
# langchain 看到这个类型之后，不会把它暴露给 LLM，而是直接传递给工具函数。
# 并且，langchain 还会在工具被调用时自动把当前的运行时信息注入进去。

@tool
def get_user_location(config: RunnableConfig) -> str:
    """
    Retrieve user information based on user ID.
    这里使用了新的 RunnableConfig 类型来获取运行时的配置信息。
    其中直接使用通用的字典结构 config 来获取需要的 user_id。
    不再需要显式地定义一个 Context 类。
    """
    # 从 config 中获取 user_id
    # 调用时需要通过 .invoke(..., config={"configurable": {"user_id": "1"}}) 传入
    configuration = config.get("configurable", {})
    user_id = configuration.get("user_id")
    return "Florida" if user_id == "1" else "SF"

## 3. 配置大模型

In [5]:
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0,
)

## 4. 定义响应格式
如果你需要代理响应匹配特定模式，也可以选择定义结构化响应格式。

In [6]:
# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

## 5. 添加内存
给代理添加内存以维持交互间的状态。这使代理能够记住之前的对话和上下文。

In [7]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

## 6. 创建并运行代理
组装之前的组件。

In [14]:
from langchain.agents.structured_output import ToolStrategy
# from langchain.agents import create_agent
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage

agent = create_react_agent(
    model=model,
    tools=[get_user_location, get_weather_for_location],
    prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
    # response_format=ResponseFormat,
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1", "user_id": "2"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    # context=Context(user_id="1")
)

# print(response['structured_response'])
print(response["messages"][-1].content)
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    # context=Context(user_id="1")
)

print(response["messages"][-1].content)
# print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

/var/folders/qz/4hkyy7hs45d18375s1sjjyw00000gp/T/ipykernel_7260/1517482675.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


Well, it looks like San Francisco is also living up to a sunny reputation! The forecast shows it's always sunny there too - I guess you could say the weather is "SF-abulous"! ☀️

But seriously, if you're in San Francisco, you're in for a bright day. Just remember that even when it's sunny, you might want to bring a light jacket - you know how SF can be "chill" even on sunny days!
You're welcome! I'm "SF-astic" that I could help! If you ever need another weather check, just "fog-et" about the fog and ask me anytime! Have a "golden" day in the Golden Gate city! 🌉☀️
